In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/IncidentDescriptions.tsv'
df = pd.read_csv(file_path, sep='\t')

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
print("Loaded records:", len(df))

Loaded records: 1094


In [ ]:
def clean_text(text):
    """Standardize whitespace and remove HTML remnants"""
    if isinstance(text, str):
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return None

In [ ]:
# data parsing logic
# import re
# from bs4 import BeautifulSoup

def parse_incident_html(raw_html: str):
    """
    Parse an incident's HTML field to extract key information.
    Works for both structured tables and freeform HTML text.
    """
    if not raw_html or not isinstance(raw_html, str):
        return {
            "Account": None,
            "Report_Time": None,
            "Resource_Type": None,
            "Resource": None,
            "Current_State": None,
            "Monitor_DisplayName": None,
            "Title": None,
            "Message": None,
            "Icm_OwningTeamId": None,
            "Forest": None,
            "WorkloadName": None,
            "TeamName": None,
            "Troubleshooting_Text": None,
            "Additional_Info": None,
            "Problem": None,
        }

    soup = BeautifulSoup(raw_html, "html.parser")
    text = soup.get_text(" ", strip=True)

    result = {
        "Account": None,
        "Report_Time": None,
        "Resource_Type": None,
        "Resource": None,
        "Current_State": None,
        "Monitor_DisplayName": None,
        "Title": None,
        "Message": None,
        "Icm_OwningTeamId": None,
        "Forest": None,
        "WorkloadName": None,
        "TeamName": None,
        "Troubleshooting_Text": None,
        "Additional_Info": None,
        "Problem": None,
    }

    for table in soup.find_all("table"):
        for row in table.find_all("tr"):
            cells = [cell.get_text(" ", strip=True) for cell in row.find_all(["th", "td"])]
            if len(cells) >= 2:
                key, value = cells[0], cells[1]
                key_lower = key.lower()
                if "account" in key_lower:
                    result["Account"] = value
                elif "report time" in key_lower:
                    result["Report_Time"] = value
                elif "resource type" in key_lower:
                    result["Resource_Type"] = value
                elif "resource" in key_lower:
                    result["Resource"] = value
                elif "current state" in key_lower:
                    result["Current_State"] = value
                elif "monitor.displayname" in key_lower or "monitor" in key_lower:
                    result["Monitor_DisplayName"] = value
                elif "title" in key_lower:
                    result["Title"] = value
                elif "message" in key_lower:
                    result["Message"] = value
                elif "icm.owningteamid" in key_lower:
                    result["Icm_OwningTeamId"] = value
                elif "forest" in key_lower:
                    result["Forest"] = value
                elif "workloadname" in key_lower:
                    result["WorkloadName"] = value
                elif "teamname" in key_lower:
                    result["TeamName"] = value

    if not result["Account"]:
        match = re.search(r"Account:\s*([A-Za-z0-9_\-]+)", text)
        if match:
            result["Account"] = match.group(1)

    if not result["Report_Time"]:
        match = re.search(r"Report Time:\s*([\d\-:TZ ]+)", text)
        if match:
            result["Report_Time"] = match.group(1)

    if not result["Resource"]:
        match = re.search(r"Resource:\s*([^\s]+)", text)
        if match:
            result["Resource"] = match.group(1)

    if not result["Title"]:
        match = re.search(r"\[S\d+.*?\].*?detected.*?(?=\s|$)", text)
        if match:
            result["Title"] = match.group(0)

    if not result["Message"]:
        match = re.search(r"Resource is [^.]+(\.[^.]*)?", text)
        if match:
            result["Message"] = match.group(0)

    trouble = re.findall(r"(Troubleshooting:|How to read this page:).*?(?=https|$)", text)
    if trouble:
        result["Troubleshooting_Text"] = " ".join(t.strip() for t in trouble)

    urls = re.findall(r"https?://[^\s]+", text)
    if urls:
        result["Additional_Info"] = " | ".join(urls[:5])

    if not result["Problem"]:
        for field in ["Title", "Message"]:
            if result[field]:
                result["Problem"] = result[field]
                break

    return result

In [ ]:
parsed_records = []
for _, row in df.iterrows():
    parsed = parse_incident_html(row["IncidentDescription"])
    parsed["Incident_ID"] = row["IncidentId"]
    parsed_records.append(parsed)

cleaned_df = pd.DataFrame(parsed_records)

print("Parsed records:", len(cleaned_df))

Parsed records: 1094


In [ ]:
display(cleaned_df)

,Account,Report_Time,Resource_Type,Resource,Current_State,Monitor_DisplayName,Title,Message,Icm_OwningTeamId,Forest,WorkloadName,TeamName,Troubleshooting_Text,Additional_Info,Problem,Incident_ID
0,Monitor: S360KPISLAMonitorV2 ( view monitor ),2025-07-21 15:30:26Z,Resource: _yifen_[SFI-TI3.1.1] Soft Delete unu...,_yifen_[SFI-TI3.1.1],None,Hybrid,[S360KPISLAMonitorV2] has detected a S360 item...,Resource is unhealthy. Expression: S360KPISLA ...,EXCHANGE\ECCLSPassiveMonitorTraining,None,Exchange,ECCLSPassiveMonitorTraining,How to read this page:,None,[S360KPISLAMonitorV2] has detected a S360 item...,659614427
1,Monitor: S360KPISLAMonitorV2 ( view monitor ),2025-07-18 17:55:06Z,Resource: _kakhidr_[SFI-PS4.4] M365 Security R...,_kakhidr_[SFI-PS4.4],None,Hybrid,[S360KPISLAMonitorV2] has detected a S360 item...,Resource is unhealthy. Expression: S360KPISLA ...,EXCHANGE\ECCLSPassiveMonitorTraining,None,Exchange,ECCLSPassiveMonitorTraining,How to read this page:,None,[S360KPISLAMonitorV2] has detected a S360 item...,658219164
2,None,None,None,None,None,None,None,None,None,None,None,None,Troubleshooting:,https://dev.azure.com/O365Exchange/,None,658170775
3,Monitor: S360KPISLAMonitorV2 ( view monitor ),2025-07-18 15:42:26Z,Resource: _jichen3_[SFI-TI3.1.1] Soft Delete u...,_jichen3_[SFI-TI3.1.1],None,Hybrid,[S360KPISLAMonitorV2] has detected a S360 item...,Resource is unhealthy. Expression: S360KPISLA ...,EXCHANGE\ECCLSPassiveMonitorTraining,None,Exchange,ECCLSPassiveMonitorTraining,How to read this page:,None,[S360KPISLAMonitorV2] has detected a S360 item...,658173276
4,None,None,None,None,None,None,None,None,None,None,None,None,Troubleshooting:,https://aka.ms/OwaEsc: | https://dev.azure.com...,None,674397793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,None,None,None,unhealthy,None,"Violated, OutOfSlaMessageCount Greater than 50...",Transport Mailbox Delivery (QMBA) of tenant in...,Resource is unhealthy. Evaluated value: 0. Loo...,EXCHANGE\TenantNotification,deup281,Exchange,Tenant Notification,None,https://o365exchange.visualstudio.com/O365%20C...,Transport Mailbox Delivery (QMBA) of tenant in...,656168775
1090,None,None,None,unhealthy,None,"Violated, OutOfSlaMessageCount Greater than 50...",Transport Mailbox Delivery (QMBA) of tenant in...,Resource is unhealthy. Evaluated value: 0. Loo...,EXCHANGE\TenantNotification,deup281,Exchange,Tenant Notification,None,https://o365exchange.visualstudio.com/O365%20C...,Transport Mailbox Delivery (QMBA) of tenant in...,656107031
1091,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,656179060
1092,None,None,None,None,None,None,None,None,None,None,None,None,Troubleshooting: Troubleshooting:,https://proofpoint.my.site.com/community/s/pro...,None,656164099


In [ ]:
output_path = "cleaned_incidents.csv"
cleaned_df.to_csv(output_path, index=False)
print(f"Cleaned data exported to: {output_path}")

Cleaned data exported to: cleaned_incidents.csv


In [ ]:
from google.colab import files
files.download("cleaned_incidents.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>